In [4]:
import numpy as np
import torch

In [11]:
a=np.random.RandomState(123).randint(1, 100, (12,12))
print(a)
np.dstack(np.unravel_index(np.argsort(a.ravel()), (12, 12)))[0][::-1]

[[67 93 99 18 84 58 87 98 97 48 74 33]
 [47 97 26 84 79 37 97 81 69 50 56 68]
 [ 3 85 40 67 85 48 62 49  8 93 53 98]
 [86 95 28 35 98 77 41  4 70 65 76 35]
 [59 11 23 78 19 16 28 31 53 71 27 81]
 [ 7 15 76 55 72  2 44 59 56 26 51 85]
 [57 50 13 19 82  2 52 45 49 57 92 50]
 [87  4 68 12 22 90 99  4 12  4 95  7]
 [10 88 15 84 71 13 55 28 39 18 62 75]
 [66 48 17  6 87 47 16 60 41 26 46 50]
 [ 1 36 30  2 84 69 31  8 94 61 66 77]
 [68 45 52  8 89 71 14 29 64 85 37 97]]


array([[[10,  0],
        [ 6,  5],
        [10,  3],
        [ 5,  5],
        [ 2,  0],
        [ 7,  7],
        [ 7,  1],
        [ 3,  7],
        [ 7,  9],
        [ 9,  3],
        [ 5,  0],
        [ 7, 11],
        [10,  7],
        [ 2,  8],
        [11,  3],
        [ 8,  0],
        [ 4,  1],
        [ 7,  8],
        [ 7,  3],
        [ 6,  2],
        [ 8,  5],
        [11,  6],
        [ 5,  1],
        [ 8,  2],
        [ 9,  6],
        [ 4,  5],
        [ 9,  2],
        [ 8,  9],
        [ 0,  3],
        [ 4,  4],
        [ 6,  3],
        [ 7,  4],
        [ 4,  2],
        [ 9,  9],
        [ 1,  2],
        [ 5,  9],
        [ 4, 10],
        [ 4,  6],
        [ 3,  2],
        [ 8,  7],
        [11,  7],
        [10,  2],
        [ 4,  7],
        [10,  6],
        [ 0, 11],
        [ 3,  3],
        [ 3, 11],
        [10,  1],
        [ 1,  5],
        [11, 10],
        [ 8,  8],
        [ 2,  2],
        [ 9,  8],
        [ 3,  6],
        [ 5,  6],
        [ 